In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import csv
from collections import Counter

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Summary

In this notebook we contruct additional features using the attributes that is available in the database of Leuven University Fund (LUF). <br>
The features are used by classifier that predicts if a donor donates for the first time, how likely is it that he will give a second time? <br>

The features constructed are: <br>
- Construct binary features for attributes that has many missing values such as Title, Academic Title, Birthplace, Firstname, Nationality, Bank account of donor known etc
- Age of first donation OR Using binary features - Age category of first donation
- Is the donation made from Belgium? Or not?
- Does the donor speak Netherlands Or not?
- Sum of first donation
- Province in Belgium for the first donation
- Year, Month, Day of first donation
- 3 binary features for fund, chair, and other credits
- No of days between donation date & date of submission of thank you letter including the date itself as another feature
- Use Partner Roles, Partner Relations, and Marketing Attributes as features

The target class is named <b>'IS_MULTI_DONOR'</b> and this is set to 1 if the donor has donated more than once to LUF, else it is set to 0. <br>

The data required for this notebook is generated using MySQL and is named as question2.csv. <br>
In case if a donor donates multiple times to LUF, we only consider the attributes during first donation. <br>

The csv file that is generated in this notebook is also used in 'Donor Feature Analysis.ipynb' notebook for analysing the influence of each feature in being a Multi-donor.

In [3]:
question2 = pd.read_csv('sqldata/question2.csv',dtype = str)
print(question2.shape)
question2.head()

(130934, 32)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,CHANGED_BY,ZZBANKN_BET,ZZGESTRUCT_MED,CREDIT_CATEGORY,CREDIT_DESCRIPTION,FISCAAL_ATTEST,DANKBRIEF,FISCJAAR,GJAHR,ORGEENHEID
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,NaN,NaN,00003,dienst Alumni,NaN,03,2004,2004,NaN
1,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,BE90431004633132,NaN,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN
2,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,NaN,NaN,00003,dienst Alumni,NaN,03,2007,2007,NaN
3,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,BE90431004633132,NaN,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN
4,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,BE90431004633132,NaN,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN


In [4]:
#Check for inconsistency among the data. One inconsistency could be if the donation sum is less than 0
question2['ZZAMNT_PAYED'] = question2['ZZAMNT_PAYED'].astype('float64')
question2[question2['ZZAMNT_PAYED'] < 0]

,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,CHANGED_BY,ZZBANKN_BET,ZZGESTRUCT_MED,CREDIT_CATEGORY,CREDIT_DESCRIPTION,FISCAAL_ATTEST,DANKBRIEF,FISCJAAR,GJAHR,ORGEENHEID
125563,0201222693,1,0,NaN,N,BE,NaN,X,NaN,X,...,U0079331,NaN,400001501822,00001,Fonds Digestieve Oncologie,02,01,2017,2017,KLINISCHE DIGESTIEVE ONCOLOGIE
125565,0201222694,1,0,NaN,NaN,NaN,NaN,X,X,NaN,...,U0079331,NaN,400001501822,00001,Fonds Digestieve Oncologie,02,01,2017,2017,KLINISCHE DIGESTIEVE ONCOLOGIE


As you can see there are two entries where donation amount is -675 Euro. This is also a joint donation. This data needs to be removed.

In [5]:
question2 = question2[question2['ZZAMNT_PAYED'] >= 0]
question2.reset_index(drop=True,inplace=True)
print(question2.shape)
question2.head()

(130932, 32)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,CHANGED_BY,ZZBANKN_BET,ZZGESTRUCT_MED,CREDIT_CATEGORY,CREDIT_DESCRIPTION,FISCAAL_ATTEST,DANKBRIEF,FISCJAAR,GJAHR,ORGEENHEID
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,NaN,NaN,00003,dienst Alumni,NaN,03,2004,2004,NaN
1,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,BE90431004633132,NaN,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN
2,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,NaN,NaN,00003,dienst Alumni,NaN,03,2007,2007,NaN
3,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,BE90431004633132,NaN,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN
4,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,U0051574,BE90431004633132,NaN,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN


In [6]:
#Normalize the sum: If two person payed 1000 Euro, then each person's normalized sum is 500 Euro
df = pd.DataFrame.from_dict(Counter((question2['ORDER_GUID'])), orient='index').reset_index()
df = df.rename(columns={'index':'ORDER_GUID', 0:'ORDER_GUID_COUNT'})
question2 = pd.merge(question2,df,on='ORDER_GUID',how='inner')
question2['ZZAMNT_PAYED'] = question2['ZZAMNT_PAYED'].astype('float64')
question2['ORDER_GUID_COUNT'] = question2['ORDER_GUID_COUNT'].astype('int32')
print(question2['ZZAMNT_PAYED'].sum())
question2['ZZAMNT_PAYED_NORM'] = question2['ZZAMNT_PAYED']/question2['ORDER_GUID_COUNT']
print(question2['ZZAMNT_PAYED_NORM'].sum())
print(question2.shape)

138439687.52
130564581.12999998
(130932, 34)


In [7]:
#Calculate all the statistics: Actually just the Total Count is needed for this exercise
order_subset = question2[['PARTNER','ZZAMNT_PAYED_NORM']]
aggregate = pd.DataFrame(group.describe().rename(columns={'ZZAMNT_PAYED_NORM':name}).squeeze()
                         for name, group in order_subset.groupby('PARTNER'))
aggregate['PARTNER'] = aggregate.index
aggregate = aggregate.reset_index(drop=True)
aggregate = aggregate.rename(columns={'count':'DONATION_COUNT',
                                      'mean':'DONATION_MEAN',
                                      'std':'DONATION_STD',
                                      'min':'DONATION_MIN',
                                      '25%':'DONATION_25',
                                      '50%':'DONATION_50',
                                      '75%':'DONATION_75',
                                      'max':'DONATION_MAX'})
                                      
front = aggregate['PARTNER']
aggregate.drop(labels=['PARTNER'], axis=1,inplace = True)
aggregate.insert(0, 'PARTNER', front)
print(aggregate.shape)
aggregate.head()

(56897, 9)


,PARTNER,DONATION_COUNT,DONATION_MEAN,DONATION_STD,DONATION_MIN,DONATION_25,DONATION_50,DONATION_75,DONATION_MAX
0,0200000002,7.0,2186.277143,735.028946,1000.0,1901.970,2500.0,2500.0,3000.00
1,0200000007,13.0,14058.253077,20169.415559,1250.0,4399.100,5750.0,11500.0,63563.29
2,0200000010,8.0,575.000000,265.921578,100.0,500.000,500.0,725.0,1000.00
3,0200000013,19.0,12567.816842,10158.236431,1000.0,3907.125,9110.0,24317.5,32000.00
4,0200000024,1.0,20000.000000,NaN,20000.0,20000.000,20000.0,20000.0,20000.00


In [8]:
aggregate = aggregate[['PARTNER','DONATION_COUNT']]
question2 = pd.merge(question2, aggregate, on='PARTNER', how='inner')
print(question2.shape)
question2.head()

(130932, 35)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,CREDIT_CATEGORY,CREDIT_DESCRIPTION,FISCAAL_ATTEST,DANKBRIEF,FISCJAAR,GJAHR,ORGEENHEID,ORDER_GUID_COUNT,ZZAMNT_PAYED_NORM,DONATION_COUNT
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,dienst Alumni,NaN,03,2004,2004,NaN,1,1000.00,7.0
1,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN,1,2500.00,7.0
2,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,dienst Alumni,NaN,03,2007,2007,NaN,1,3000.00,7.0
3,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN,1,1303.94,7.0
4,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN,1,2500.00,7.0


Sort Values according to PARTNER & DOCUMENT DATE

In [9]:
question2.sort_values(['PARTNER', 'ZLUF_DOCDATE'], ascending=[1,1])
question2.reset_index(drop=True, inplace=True)
print(question2.shape)
question2.head()

(130932, 35)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,CREDIT_CATEGORY,CREDIT_DESCRIPTION,FISCAAL_ATTEST,DANKBRIEF,FISCJAAR,GJAHR,ORGEENHEID,ORDER_GUID_COUNT,ZZAMNT_PAYED_NORM,DONATION_COUNT
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,dienst Alumni,NaN,03,2004,2004,NaN,1,1000.00,7.0
1,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN,1,2500.00,7.0
2,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,dienst Alumni,NaN,03,2007,2007,NaN,1,3000.00,7.0
3,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN,1,1303.94,7.0
4,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,Faculteit Rechtsgeleerdheid,NaN,03,2014,2014,NaN,1,2500.00,7.0


Only select the entry with first donation

In [10]:
question2 = question2.sort_values(['PARTNER', 'ZLUF_DOCDATE'], ascending=[1,1]).groupby('PARTNER').head(1)
question2.reset_index(drop=True, inplace=True)
print(question2.shape)
question2.head()

(56897, 35)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,CREDIT_CATEGORY,CREDIT_DESCRIPTION,FISCAAL_ATTEST,DANKBRIEF,FISCJAAR,GJAHR,ORGEENHEID,ORDER_GUID_COUNT,ZZAMNT_PAYED_NORM,DONATION_COUNT
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,dienst Alumni,NaN,03,2004,2004,NaN,1,1000.0,7.0
1,0200000007,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,dep. Farmaceutische Wetenschappen,NaN,03,2010,2010,NaN,1,4000.0,13.0
2,0200000010,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00001,Science@Leuven Fund,NaN,03,2009,2009,NaN,1,100.0,8.0
3,0200000013,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00003,Fac. Economie en Bedrijfswetenschappen,NaN,03,2012,2012,NaN,1,5000.0,19.0
4,0200000024,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00001,KU Leuven Erfgoedfonds,01,02,2018,2018,DIENST ALUMNI,1,20000.0,1.0


#### Create binary features from columns that have many missing values or columns that are categorical

In [11]:
question2['ISK_PERSON'] = (question2['TYPE'] == '1').astype(int)                #Is donor, person or an organization
question2['ISK_BIRTHPL'] = question2['BIRTHPL'].notnull().astype(int)           #Birthplace
question2['ISK_BU_LANGU'] = question2['BU_LANGU'].notnull().astype(int)         #Native Language
question2['ISK_CNT_ORIGIN'] = question2['CNT_ORIGIN'].notnull().astype(int)     #Country of Origin
question2['ISK_NATIONALITY'] = question2['NATIONALITY'].notnull().astype(int)   #Nationality
question2['ISK_VOORNAAM'] = question2['ZZGEEN_VOORNAAM'].notnull().astype(int)  #Firstname
question2['XSEXF'] = question2['XSEXF'].notnull().astype(int)                   #Categorical Sex Female (Overwrite)
question2['XSEXM'] = question2['XSEXM'].notnull().astype(int)                   #Categorical Sex Male   (Overwrite)
question2['ISK_LANGU_CORR'] = question2['LANGU_CORR'].notnull().astype(int)     #Correspondance Language
question2['ISK_MARST'] = question2['MARST'].isin(['1','2','3','4','5','6','7','8']).astype(int) #Maritial Status
question2['ISK_TITLE'] = question2['TITLE'].notnull().astype(int)               #Title
question2['ISK_ACAT'] = question2['TITLE_ACADEMDIC'].notnull().astype(int)      #Academic Title
question2['ISK_CNT_BE'] = (question2['COUNTRY'] == 'BE').astype(int)            #Is Country of donation = BE
question2['ISK_COMM_LET'] = (question2['DEFLT_COMM'] == 'LET').astype(int)      #Is method of communication = LET

#Binary features for first donations
question2['ISK_FD_SHARED'] = (question2['ORDER_GUID_COUNT'] > 1).astype(int)    #Is first donation shared?
question2['ISK_FD_CAT_1'] = ((question2['ZZAMNT_PAYED_NORM'] >= 0) & (question2['ZZAMNT_PAYED_NORM'] <= 1249)).astype(int)
question2['ISK_FD_CAT_2'] = ((question2['ZZAMNT_PAYED_NORM'] >= 1250) & (question2['ZZAMNT_PAYED_NORM'] <= 4999)).astype(int)
question2['ISK_FD_CAT_3'] = ((question2['ZZAMNT_PAYED_NORM'] >= 5000) & (question2['ZZAMNT_PAYED_NORM'] <= 14999)).astype(int)
question2['ISK_FD_CAT_4'] = ((question2['ZZAMNT_PAYED_NORM'] >= 15000) & (question2['ZZAMNT_PAYED_NORM'] <= 24999)).astype(int)
question2['ISK_FD_CAT_5'] = ((question2['ZZAMNT_PAYED_NORM'] >= 25000) & (question2['ZZAMNT_PAYED_NORM'] <= 49999)).astype(int)
question2['ISK_FD_CAT_6'] = ((question2['ZZAMNT_PAYED_NORM'] >= 50000)).astype(int)
question2['ISK_FD_BANKN'] = question2['ZZBANKN_BET'].notnull().astype(int)      #Is the bank account of donor known
question2['ISK_FD_VGS'] = question2['ZZGESTRUCT_MED'].notnull().astype(int)     #Did the donor provide VGS Statement
question2['ISK_FD_TAX_NE'] = (question2['FISCAAL_ATTEST'] == '02').astype(int)  #Tax not eligible
question2['ISK_FD_TYL_NN'] = (question2['DANKBRIEF'] == '01').astype(int)       #Thank you letter not needed

print(question2.shape)
question2.head()

(56897, 58)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,ISK_FD_CAT_1,ISK_FD_CAT_2,ISK_FD_CAT_3,ISK_FD_CAT_4,ISK_FD_CAT_5,ISK_FD_CAT_6,ISK_FD_BANKN,ISK_FD_VGS,ISK_FD_TAX_NE,ISK_FD_TYL_NN
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0200000007,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0200000010,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0200000013,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0200000024,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,1,0,0,1,0,0,0


#### Province

In [12]:
question2['POST_CODE'] = question2['POST_CODE'].fillna('3000')#Fill 121 NaN values to Leuven postcode

#If the country of donation is BE, then consider the post codes
question2['BE_POST'] = np.where(question2['ISK_CNT_BE'], question2['POST_CODE'], 0).astype('int32')

#Binary features for each province in Belgium
question2['ISK_PRO_BCR'] = ((question2['BE_POST'] >= 1000) & (question2['BE_POST'] <= 1299)).astype(int) #Brussels Capital Region
question2['ISK_PRO_WB'] = ((question2['BE_POST'] >= 1300) & (question2['BE_POST'] <= 1499)).astype(int)  #Waloon Brabant
question2['ISK_PRO_FB'] = (((question2['BE_POST'] >= 1500) & (question2['BE_POST'] <= 1999)) | 
                           ((question2['BE_POST'] >= 3000) & (question2['BE_POST'] <= 3499))).astype(int)#Flemish Brabant
question2['ISK_PRO_AWP'] = ((question2['BE_POST'] >= 2000) & (question2['BE_POST'] <= 2999)).astype(int) #Antwerp
question2['ISK_PRO_LBG'] = ((question2['BE_POST'] >= 3500) & (question2['BE_POST'] <= 3999)).astype(int) #Limburg
question2['ISK_PRO_LGE'] = ((question2['BE_POST'] >= 4000) & (question2['BE_POST'] <= 4999)).astype(int) #Liege
question2['ISK_PRO_NMU'] = ((question2['BE_POST'] >= 5000) & (question2['BE_POST'] <= 5999)).astype(int) #Namur
question2['ISK_PRO_HNT'] = (((question2['BE_POST'] >= 6000) & (question2['BE_POST'] <= 6599)) | 
                           ((question2['BE_POST'] >= 7000) & (question2['BE_POST'] <= 7999))).astype(int)#Hainaut
question2['ISK_PRO_LUX'] = ((question2['BE_POST'] >= 6600) & (question2['BE_POST'] <= 6999)).astype(int) #Luxembourg
question2['ISK_PRO_WF'] = ((question2['BE_POST'] >= 8000) & (question2['BE_POST'] <= 8999)).astype(int)  #West Flanders
question2['ISK_PRO_EF'] = ((question2['BE_POST'] >= 9000) & (question2['BE_POST'] <= 9999)).astype(int)  #East Flanders

#### Create binary features for Credits (Funds, Chairs, or Other Credits)

In [13]:
question2['ISK_CC_FD_F'] = (question2['CREDIT_CATEGORY'] == '00001').astype(int)     #Is first donation to Funds?
question2['ISK_CC_FD_CH'] = (question2['CREDIT_CATEGORY'] == '00002').astype(int)    #Is first donation to Chairs?
question2['ISK_CC_FD_OC'] = (question2['CREDIT_CATEGORY'] == '00003').astype(int)    #Is first donation to Other Credits?

#### Number of days of sending thank you letter

In [14]:
#Convert String to DateTime
question2['ZLUF_DOCDATE'] = pd.to_datetime(question2['ZLUF_DOCDATE'])
question2['ZLUF_DANKBRF'] = pd.to_datetime(question2['ZLUF_DANKBRF'])
#Create a new column that stores number of days to receive thank you letter 
#Of course, only the first donation case is considered
#Need to handle missing data & negative value (It can't be negative!)
question2['NUM_D_TYL'] = (question2['ZLUF_DANKBRF'] - question2['ZLUF_DOCDATE']).dt.days
#For missing values set it to max days
question2['NUM_D_TYL'] = question2['NUM_D_TYL'].fillna(question2['NUM_D_TYL'].max()) 
#For negative values set it to max days [If thank you letter was sent earlier than doc_date then it is wrong!]
question2['NUM_D_TYL'] = np.where(question2['NUM_D_TYL'] < 0, question2['NUM_D_TYL'].max(), question2['NUM_D_TYL'])
print(question2.shape)
question2.head()

(56897, 74)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,ISK_PRO_LGE,ISK_PRO_NMU,ISK_PRO_HNT,ISK_PRO_LUX,ISK_PRO_WF,ISK_PRO_EF,ISK_CC_FD_F,ISK_CC_FD_CH,ISK_CC_FD_OC,NUM_D_TYL
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,1,3550.0
1,0200000007,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,1,220.0
2,0200000010,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,1,0,0,98.0
3,0200000013,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,1,111.0
4,0200000024,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,1,0,0,3550.0


#### Age Category

In [15]:
question2['AGE_FIRST_DONATION'] = question2['ZLUF_DOCDATE'].dt.year - question2['ZZGEBOORTEJAAR'].astype(int)
question2['ISK_AGE_FD_CAT1'] = ((question2['AGE_FIRST_DONATION'] >= 0) & (question2['AGE_FIRST_DONATION'] < 25)).astype(int)
question2['ISK_AGE_FD_CAT2'] = ((question2['AGE_FIRST_DONATION'] >= 25) & (question2['AGE_FIRST_DONATION'] < 50)).astype(int)
question2['ISK_AGE_FD_CAT3'] = ((question2['AGE_FIRST_DONATION'] >= 50) & (question2['AGE_FIRST_DONATION'] < 75)).astype(int)
question2['ISK_AGE_FD_CAT4'] = ((question2['AGE_FIRST_DONATION'] >= 75) & (question2['AGE_FIRST_DONATION'] < 120)).astype(int)
question2['ISK_AGE_FD_CATU'] = ((question2['AGE_FIRST_DONATION'] >= 120) | (question2['AGE_FIRST_DONATION'] < 0)).astype(int)

#### Year & Month of first donation

In [16]:
question2['YEAR_FD'] = question2['ZLUF_DOCDATE'].dt.year
question2['MONTH_FD'] = question2['ZLUF_DOCDATE'].dt.month
question2['DAY_FD'] = question2['ZLUF_DOCDATE'].dt.day
question2['WEEK_FD'] = question2['ZLUF_DOCDATE'].dt.week
question2['WEEKDAY_FD'] = question2['ZLUF_DOCDATE'].dt.weekday

date_data = question2[['PARTNER','YEAR_FD','MONTH_FD','DAY_FD','WEEK_FD']]
month_data = date_data[['PARTNER','MONTH_FD']]
month_data['MONTH_FD'] = month_data['MONTH_FD'].astype(str)
month_data['MONTH_FD'] = 'ISK_FD_M_' + month_data['MONTH_FD']
month_data.head()

,PARTNER,MONTH_FD
0,0200000002,ISK_FD_M_7
1,0200000007,ISK_FD_M_11
2,0200000010,ISK_FD_M_12
3,0200000013,ISK_FD_M_2
4,0200000024,ISK_FD_M_1


In [17]:
#Binary features for month of first donation
month_fd_matrix = pd.crosstab(index=month_data.iloc[:,0],columns=month_data.iloc[:,1])
month_cols = ['PARTNER'] + ['ISK_FD_M_'+str(x+1) for x in range(12)]
month_fd_matrix['PARTNER'] = month_fd_matrix.index
month_fd_matrix = month_fd_matrix[month_cols]
month_fd_matrix.reset_index(drop=True, inplace=True)
print(month_fd_matrix.shape)
month_fd_matrix.head()

(56897, 13)


MONTH_FD,PARTNER,ISK_FD_M_1,ISK_FD_M_2,ISK_FD_M_3,ISK_FD_M_4,ISK_FD_M_5,ISK_FD_M_6,ISK_FD_M_7,ISK_FD_M_8,ISK_FD_M_9,ISK_FD_M_10,ISK_FD_M_11,ISK_FD_M_12
0,0200000002,0,0,0,0,0,0,1,0,0,0,0,0
1,0200000007,0,0,0,0,0,0,0,0,0,0,1,0
2,0200000010,0,0,0,0,0,0,0,0,0,0,0,1
3,0200000013,0,1,0,0,0,0,0,0,0,0,0,0
4,0200000024,1,0,0,0,0,0,0,0,0,0,0,0


In [18]:
year_data = date_data[['PARTNER','YEAR_FD']]
year_data['YEAR_FD'] = year_data['YEAR_FD'].astype(str)
year_data['YEAR_FD'] = 'ISK_FD_Y_' + year_data['YEAR_FD']
year_data.head()

,PARTNER,YEAR_FD
0,0200000002,ISK_FD_Y_2004
1,0200000007,ISK_FD_Y_2010
2,0200000010,ISK_FD_Y_2009
3,0200000013,ISK_FD_Y_2012
4,0200000024,ISK_FD_Y_2018


In [19]:
#Binary features for year of first donation
year_fd_matrix = pd.crosstab(index=year_data.iloc[:,0],columns=year_data.iloc[:,1])
year_cols = ['PARTNER']+['ISK_FD_Y_'+str(x) for x in range(question2['ZLUF_DOCDATE'].dt.year.min(),
                                                           question2['ZLUF_DOCDATE'].dt.year.max()+1)]
year_fd_matrix['PARTNER'] = year_fd_matrix.index
year_fd_matrix = year_fd_matrix[year_cols]
year_fd_matrix.reset_index(drop=True, inplace=True)
print(year_fd_matrix.shape)
year_fd_matrix.head()

(56897, 18)


YEAR_FD,PARTNER,ISK_FD_Y_2002,ISK_FD_Y_2003,ISK_FD_Y_2004,ISK_FD_Y_2005,ISK_FD_Y_2006,ISK_FD_Y_2007,ISK_FD_Y_2008,ISK_FD_Y_2009,ISK_FD_Y_2010,ISK_FD_Y_2011,ISK_FD_Y_2012,ISK_FD_Y_2013,ISK_FD_Y_2014,ISK_FD_Y_2015,ISK_FD_Y_2016,ISK_FD_Y_2017,ISK_FD_Y_2018
0,0200000002,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0200000007,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0200000010,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0200000013,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0200000024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [20]:
#Merge month of fd & year of fd matrices with question2 data
time_matrix_merge = pd.merge(month_fd_matrix,year_fd_matrix,on='PARTNER',how='inner')
question2 = pd.merge(question2,time_matrix_merge,on='PARTNER',how='inner')
question2.columns = question2.columns.str.replace('_x','')
question2.columns = question2.columns.str.replace('_y','')
print(question2.shape)
question2.head()

(56897, 114)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,ISK_FD_Y_2009,ISK_FD_Y_2010,ISK_FD_Y_2011,ISK_FD_Y_2012,ISK_FD_Y_2013,ISK_FD_Y_2014,ISK_FD_Y_2015,ISK_FD_Y_2016,ISK_FD_Y_2017,ISK_FD_Y_2018
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0200000007,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0200000010,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0200000013,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0200000024,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,1


#### ROLES (from BUT100)

In [21]:
partner_roles = pd.read_csv('SQLDATA/PYTHON_OUTPUTS/Roles_p_r_matrix.csv',dtype = str)
print(partner_roles.shape)
#Not all roles are needed as some of them are all 0's and these are screened
required_cols = ['BUP003','Z00001','Z00002','Z00003','Z00004','Z00005','Z00013','Z00016','Z00017','Z00018','Z00019','Z00020',
                 'ZDONOR','ZERFL2','ZERFLA','ZKBOO','ZKBOV','ZKLRD','ZOVERL','ZPROSP','ZSCHSP','ZSCHTE','ZSTOPG']
partner_roles = partner_roles[['PARTNER']+required_cols]
partner_roles.head()

(72114, 29)


,PARTNER,BUP003,Z00001,Z00002,Z00003,Z00004,Z00005,Z00013,Z00016,Z00017,...,ZERFL2,ZERFLA,ZKBOO,ZKBOV,ZKLRD,ZOVERL,ZPROSP,ZSCHSP,ZSCHTE,ZSTOPG
0,0200000002,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0200000004,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0200000005,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0200000006,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0200000007,0,0,0,0,0,1,0,0,1,...,0,0,1,0,1,0,0,0,0,0


In [22]:
donor_list = pd.read_csv('SQLDATA/donor_list.csv',dtype = str)
print(donor_list.shape)
#but100 = pd.read_csv('sqldata/20190102_BUT100.csv',dtype = str, encoding='latin-1',sep=';')
#rltypes = list(set(but100['RLTYP']))
donor_roles = pd.merge(partner_roles, donor_list, on='PARTNER', how='inner')
donor_roles.reset_index(drop=True,inplace=True)
print(donor_roles.shape)
donor_roles.head()

(56897, 1)
(56897, 24)


,PARTNER,BUP003,Z00001,Z00002,Z00003,Z00004,Z00005,Z00013,Z00016,Z00017,...,ZERFL2,ZERFLA,ZKBOO,ZKBOV,ZKLRD,ZOVERL,ZPROSP,ZSCHSP,ZSCHTE,ZSTOPG
0,0200000002,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0200000007,0,0,0,0,0,1,0,0,1,...,0,0,1,0,1,0,0,0,0,0
2,0200000010,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
3,0200000013,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,0200000024,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [23]:
question2 = pd.merge(question2, donor_roles, on='PARTNER', how='inner')
print(question2.shape)
question2.head()

(56897, 137)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,ZERFL2,ZERFLA,ZKBOO,ZKBOV,ZKLRD,ZOVERL,ZPROSP,ZSCHSP,ZSCHTE,ZSTOPG
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0200000007,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0200000010,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0200000013,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0200000024,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0


#### RELATIONS (from BUT050 & BUT051)

In [24]:
partner_relations = pd.read_csv('SQLDATA/PYTHON_OUTPUTS/Relations_p_r_matrix.csv',dtype = str)
#Drop these columns as donors (not partners) have less than 10 value_counts:
        #'REL_Z00017', 'REL_ZSECR', 'REL_Z00019'
partner_relations = partner_relations.drop(['Unnamed: 0','REL_Z00017', 'REL_ZSECR', 'REL_Z00019'],axis=1)
print(partner_relations.shape)
partner_relations.head()

(72638, 9)


,PARTNER,REL_ORGCP,REL_BUR011,REL_BUR021,REL_BUR023,REL_BUR998,REL_Z00001,REL_Z00016,REL_Z00018
0,0200000002,0,1,0,0,0,0,0,0
1,0200000004,0,0,0,0,0,0,0,0
2,0200000005,0,0,0,0,0,0,0,0
3,0200000006,0,0,0,0,0,0,0,0
4,0200000007,2,0,0,0,0,0,0,0


In [25]:
donor_relations = pd.merge(partner_relations, donor_list, on='PARTNER', how='inner')
donor_relations.reset_index(drop=True,inplace=True)
print(donor_relations.shape)
donor_relations.head()

(56897, 9)


,PARTNER,REL_ORGCP,REL_BUR011,REL_BUR021,REL_BUR023,REL_BUR998,REL_Z00001,REL_Z00016,REL_Z00018
0,0200000002,0,1,0,0,0,0,0,0
1,0200000007,2,0,0,0,0,0,0,0
2,0200000010,0,0,0,0,0,0,0,0
3,0200000013,1,1,0,0,0,0,0,0
4,0200000024,0,0,0,0,0,0,0,0


In [26]:
question2 = pd.merge(question2, donor_relations, on='PARTNER', how='inner')
print(question2.shape)
question2.head()

(56897, 145)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,ZSCHTE,ZSTOPG,REL_ORGCP,REL_BUR011,REL_BUR021,REL_BUR023,REL_BUR998,REL_Z00001,REL_Z00016,REL_Z00018
0,0200000002,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0200000007,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,2,0,0,0,0,0,0,0
2,0200000010,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0200000013,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,1,0,0,0,0,0,0
4,0200000024,2,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Marketing Attributes

In [27]:
partner_mktg_attributes = pd.read_csv('SQLDATA/PYTHON_OUTPUTS/marketing_attributes_matrix.csv',dtype = str)
#Drop these columns as donors (not partners) have less than 10 value_counts:
        #'ZLUF_INTERESSELEGAAT_OTHER', 'ZLUF_SCHENKINTERESSE_SOCIALSERVICE', 'ZLUF_SCHENKINTERESSE_SPECIFICDISEASE'
partner_mktg_attributes = partner_mktg_attributes.drop(['Index','ZLUF_INTERESSELEGAAT_OTHER', 
                                                        'ZLUF_SCHENKINTERESSE_SOCIALSERVICE', 
                                                        'ZLUF_SCHENKINTERESSE_SPECIFICDISEASE'],axis=1)
print(partner_mktg_attributes.shape)
partner_mktg_attributes.head()

(71655, 21)


,PARTNER,ZLUF_AMBT_DOCTOR,ZLUF_AMBT_MASTERorLAWYER,ZLUF_AMBT_NOTARY,ZLUF_CAMPAGNE_BOARD,ZLUF_CAMPAGNE_DONOR,ZLUF_CAMPAGNE_PROSPECT,ZLUF_CAMPAGNE_RELATIONSHIP,ZLUF_INTERESSELEGAAT_NOTARY,ZLUF_INTERESSELEGAAT_REACHED_LUF,...,ZLUF_KERKELIJK,ZLUF_LIJST,ZLUF_ORG_ALS_PERSOON,ZLUF_ORG_VERENIGING,ZLUF_SCHENKINTERESSE_ARTnCULTURE,ZLUF_SCHENKINTERESSE_CHILDREN,ZLUF_SCHENKINTERESSE_DEVCOOP,ZLUF_SCHENKINTERESSE_HEALTHCARE,ZLUF_SCHENKINTERESSE_HERITAGE,ZLUF_SCHENKINTERESSE_SCHOLARSHIPS
0,0200000002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0200000007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0200000010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0200000013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0200000024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
donor_mktg_attributes = pd.merge(partner_mktg_attributes, donor_list, on='PARTNER', how='inner')
donor_mktg_attributes.reset_index(drop=True,inplace=True)
print(donor_mktg_attributes.shape)
donor_mktg_attributes.head()

(56866, 21)


,PARTNER,ZLUF_AMBT_DOCTOR,ZLUF_AMBT_MASTERorLAWYER,ZLUF_AMBT_NOTARY,ZLUF_CAMPAGNE_BOARD,ZLUF_CAMPAGNE_DONOR,ZLUF_CAMPAGNE_PROSPECT,ZLUF_CAMPAGNE_RELATIONSHIP,ZLUF_INTERESSELEGAAT_NOTARY,ZLUF_INTERESSELEGAAT_REACHED_LUF,...,ZLUF_KERKELIJK,ZLUF_LIJST,ZLUF_ORG_ALS_PERSOON,ZLUF_ORG_VERENIGING,ZLUF_SCHENKINTERESSE_ARTnCULTURE,ZLUF_SCHENKINTERESSE_CHILDREN,ZLUF_SCHENKINTERESSE_DEVCOOP,ZLUF_SCHENKINTERESSE_HEALTHCARE,ZLUF_SCHENKINTERESSE_HERITAGE,ZLUF_SCHENKINTERESSE_SCHOLARSHIPS
0,0200000002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0200000007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0200000010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0200000013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0200000024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
question2 = pd.merge(question2, donor_mktg_attributes, on='PARTNER', how='left')
question2 = question2.fillna(0) 
print(question2.shape)
question2.head()

(56897, 165)


,PARTNER,TYPE,ZZGEBOORTEJAAR,BIRTHPL,BU_LANGU,CNT_ORIGIN,NATIONALITY,ZZGEEN_VOORNAAM,XSEXF,XSEXM,...,ZLUF_KERKELIJK,ZLUF_LIJST,ZLUF_ORG_ALS_PERSOON,ZLUF_ORG_VERENIGING,ZLUF_SCHENKINTERESSE_ARTnCULTURE,ZLUF_SCHENKINTERESSE_CHILDREN,ZLUF_SCHENKINTERESSE_DEVCOOP,ZLUF_SCHENKINTERESSE_HEALTHCARE,ZLUF_SCHENKINTERESSE_HERITAGE,ZLUF_SCHENKINTERESSE_SCHOLARSHIPS
0,0200000002,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0200000007,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0200000010,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0200000013,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0200000024,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### TARGET CLASS

In [30]:
question2['IS_MULTI_DONOR'] = (question2['DONATION_COUNT'] > 1).astype(int)

Drop the columns that are not needed

In [31]:
question2 = question2.drop(['TYPE','ZZGEBOORTEJAAR','AGE_FIRST_DONATION','BIRTHPL', 'BU_LANGU','CNT_ORIGIN','NATIONALITY',
                            'ZZGEEN_VOORNAAM','LANGU_CORR','MARST','TITLE','TITLE_ACADEMDIC','COUNTRY','DEFLT_COMM',
                            'POST_CODE','ORDER_GUID','CREATED_BY','CHANGED_BY','ZLUF_DANKBRF','BE_POST','ZZBANKN_BET',
                            'ZZGESTRUCT_MED','CREDIT_CATEGORY','CREDIT_DESCRIPTION','FISCAAL_ATTEST','DANKBRIEF',
                            'ORGEENHEID','FISCJAAR','GJAHR','YEAR_FD','MONTH_FD','DAY_FD','WEEK_FD','WEEKDAY_FD',
                            'DONATION_COUNT','ORDER_GUID_COUNT'],axis=1)
question2.to_csv('SQLDATA/PYTHON_OUTPUTS/question2_prepared_data.csv')
print(question2.shape)
question2.head()

(56897, 130)


,PARTNER,XSEXF,XSEXM,ZZAMNT_PAYED,ZLUF_DOCDATE,ZZAMNT_PAYED_NORM,ISK_PERSON,ISK_BIRTHPL,ISK_BU_LANGU,ISK_CNT_ORIGIN,...,ZLUF_LIJST,ZLUF_ORG_ALS_PERSOON,ZLUF_ORG_VERENIGING,ZLUF_SCHENKINTERESSE_ARTnCULTURE,ZLUF_SCHENKINTERESSE_CHILDREN,ZLUF_SCHENKINTERESSE_DEVCOOP,ZLUF_SCHENKINTERESSE_HEALTHCARE,ZLUF_SCHENKINTERESSE_HERITAGE,ZLUF_SCHENKINTERESSE_SCHOLARSHIPS,IS_MULTI_DONOR
0,0200000002,0,0,1000.0,2004-07-20,1000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0200000007,0,0,4000.0,2010-11-02,4000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0200000010,0,0,100.0,2009-12-22,100.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0200000013,0,0,5000.0,2012-02-28,5000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0200000024,0,0,20000.0,2018-01-03,20000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
